# L1: Information Retrieval

In this lab you will apply basic techniques from information retrieval to implement the core of a minimalistic search engine. The data for this lab consists of a collection of app descriptions scraped from the [Google Play Store](https://play.google.com/store/apps?hl=en). From this collection, your search engine should retrieve those apps whose descriptions best match a given query under the vector space model.

## Data set

The app descriptions come in the form of a compressed [JSON](https://en.wikipedia.org/wiki/JSON) file. Start by loading this file into a [Pandas](https://pandas.pydata.org) [DataFrame](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html#dataframe).

In [3]:
import bz2
import pandas as pd

with bz2.open("app-descriptions.json.bz2") as source:
    df = pd.read_json(source)
    
# Extracting number of descriptions.
len(df)

1614

In Pandas, a DataFrame is a table with indexed rows and labelled columns of potentially different types. Data in a DataFrame can be accessed in various ways, including by row and by column. To give an example, the code in the next cell shows rows 200–204:

In [4]:
df[200:205]

,description,name
200,Introducing the best Brick Breaker game that e...,Brick Breaker Star: Space King
201,Classic Brick Game!\n\nBrick Classic is a popu...,Brick Classic - Brick Game
202,Bricks Breaker - Glow Balls is a addictive and...,Bricks Breaker - Glow Balls
203,How to play\n- The ball flies to wherever you ...,Bricks Breaker Quest
204,Fight brave soldiers from around the globe on ...,Brothers in Arms® 3


As you can see, there are two labelled columns: `name` (the name of the app) and `description` (a textual description). The code in the next cell shows how to acess fields from the description column.

In [5]:
df["description"][200:205]

200    Introducing the best Brick Breaker game that e...
201    Classic Brick Game!\n\nBrick Classic is a popu...
202    Bricks Breaker - Glow Balls is a addictive and...
203    How to play\n- The ball flies to wherever you ...
204    Fight brave soldiers from around the globe on ...
Name: description, dtype: object

## Problem 1: Preprocessing

Your first task is to implement a preprocessor for your search engine. In the vector space model, *preprocessing* refers to any kind of transformation that is applied before a text is vectorized. Here you can restrict yourself to a very simple preprocessing: tokenization, stop word removal, and lemmatization.

To implement your preprocessor, you can use [spaCy](https://spacy.io). Make sure that you read the [Linguistic annotations](https://spacy.io/usage/spacy-101#annotations) section of the spaCy&nbsp;101; that section contains all the information that you need for this problem (and more).

Implement your preprocessor by completing the skeleton code in the next cell, adding additional code as you feel necessary.

In [6]:
import spacy
 
# Loading model. Disabling loading non-necessary model components.
nlp = spacy.load("en_core_web_sm", disable = ["tagger", "parser", "ner", "textcat"])

def preprocess(text):
    
    # Initializing list with lemmas to be returned by function.
    lemmas_returned = []
    
    # Tokenization.    
    doc = nlp(text)
    
    # Appending desired lemmas of tokens to initialized list.
    for token in doc:
        # Considering token if it is not a stop word.
        if token.is_stop == False:
            # Lemmatization.
            lemma = token.lemma_
            # Appending lemma if it does not contain non-alphabetical characters.
            if lemma.isalpha() == True:
                lemmas_returned.append(lemma)
        
    # Returning remaining lemmas as strings.
    return lemmas_returned

Your implementation should conform to the following specification:

<strong>preprocess</strong> (<em>text</em>)

> Preprocesses given text by tokenizing it, removing any stop words, replacing each remaining token with its lemma (base form), and discarding all lemmas that contain non-alphabetical characters. Returns the list of remaining lemmas (represented as strings).

**Tip:** To speed up the preprocessing, you can disable loading those spaCy components that you do not need, such as the part-of-speech tagger, parser, and named entity recognizer. See [here](https://spacy.io/usage/processing-pipelines#disabling) for more information about this.

Test your implementation by running the following cell:

In [7]:
preprocess("Apple is looking at buying U.K. startup for $1 billion")

['Apple', 'look', 'buy', 'startup', 'billion']

This should give the following output:


## Problem 2: Vectorizing

Your next task is to vectorize the data – and more specifically, to map each app description to a tf–idf vector. For this you can use the [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) class from [scikit-learn](https://scikit-learn.org/stable/). Make sure to specify your preprocessor from the previous problem as the `tokenizer` &ndash; not the `preprocessor`! &ndash; for the vectorizer.

In [8]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Creating instance of TfidfVectorizer class. Initializing using own preprocessor function as the tokenizer.
vectorizer = TfidfVectorizer(tokenizer = preprocess)

# Learning vocabulary and tf-idf, returning term-document matrix. 
## Attributes of instance "vectorizer" will be adjusted.
## Matrix will be stored in X. 
X = vectorizer.fit_transform(df["description"])

Test your implementation by running the following cell:

In [9]:
# Printing shape of returned term-document matrix.
# Row indices in X refer to the app descriptions; column indices refer to the terms; values are the tf-idfs.
X.shape

(1614, 20651)

This should show the dimensions of the matrix `X` to be 1614 × 20651.

## Problem 3: Finding terms with low/high idf

Recall that the inverse document frequency (idf) of a term is the lower the more documents from a given collection the term appears in. To get a better understanding for this concept, your next task is to write code to find out which terms have the lowest/highest idf with respect to the app descriptions.

Start by sorting the terms in increasing order of idf, breaking ties by falling back on alphabetic order.

In [12]:
# Combining vocabularies and idf values in a pandas data frame.
terms = pd.DataFrame({"term": vectorizer.get_feature_names(), 
                       "idf": vectorizer.idf_})

# Sorting terms in increasing order of idf and alphabetically related to the term.
terms = terms.sort_values(by = ["idf", "term"], ascending = [True, False])

Now, print out the 10 terms at the top/end of the sorted list:

In [14]:
print(terms[-10:])
print(terms[:10])

        term       idf
16   abcsong  7.693943
14  abcmouse  7.693943
13      abcd  7.693943
11      abby  7.693943
10    abbott  7.693943
9    abbiamo  7.693943
6        aat  7.693943
5       aang  7.693943
4      aagbi  7.693943
1         aa  7.693943
          term       idf
5822      game  1.296180
11475     play  1.412611
5184   feature  1.472359
5573      free  1.584695
10192      new  1.665665
17102    world  1.781792
15419     time  1.788581
667        app  1.838871
5672       fun  1.859132
16902     well  1.872378


## Problem 4: Retrieving

To complete the search engine, your last task is to write a function that returns the most relevant app descriptions for a given query. An easy way to do solve this task is to use scikit-learn&rsquo;s [NearestNeighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html) class. That class implements unsupervised nearest neighbours learning, and allows you to easily find a predefined number of app descriptions whose vector representations are closest in distance to the query vector.

In [15]:
from sklearn.neighbors import NearestNeighbors

# Initializing class. 
## By default: n_neighbors = 5, radius = 1. 
## As a metric, we set the cosine similarity.
neigh = NearestNeighbors(metric = "cosine")
    
# Train k-nearest-neighbor-model on X.
## X is document-term-matrix for all descriptions and terms.
## Query will be compared in same vector form to each description vector of the matrix.
neigh.fit(X) 

def search(query):
    
    # Transforming query to document-term-matrix using trained vectorizer. 
    query = vectorizer.transform([query])
    
    # Identifying nearest 10 neighbors.
    ## Indices of the nearest descriptions are returned.
    idx = neigh.kneighbors(X = query, n_neighbors = 10, return_distance = False)
    
    # Returning subset of data frame with identified indices.
    return df.iloc[idx[0]]["name"]

Your implementation should conform to the following specification:

<strong>search</strong> (<em>query</em>)

> Returns the 10 app descriptions most similar (in terms of cosine similarity) to the given query as a Pandas DataFrame.

Test your implementation by running the following cell:

In [14]:
search("dodge trains")

1301                               Subway Surfers
1300                       Subway Princess Runner
1428                        Train Conductor World
998                No Humanity - The Hardest Game
1394                                   Tiny Rails
1429          Train for Animals - BabyMagica free
1168                                         Rush
1077                  Polar Flow – Sync & Analyze
1286    Strava: Track Running, Cycling & Swimming
1465              Virus War - Space Shooting Game
Name: name, dtype: object

The top hit in the list should be *Subway Surfers*.

## Problem 5: Keyword extraction

A simple method for extracting salient keywords from a document is to pick the $k$ terms with the highest tf–idf value. Your last task in this lab is to implement this method. More specifically, we ask you to implement a function `keywords` that extracts keywords from a given text.

In [15]:
def keywords(text, n = 10):
    
    # Transforming text to document-term-matrix using trained vectorizer.
    ## Matrix has one row (text is one document) and 20651 columns (terms) given by trained vectorizer.
    ## Values within matrix are the tf-idf values for each term.
    text = vectorizer.transform([text])

    # Transforming data type from csr_matrix to DataFrame.
    text = pd.DataFrame(text.toarray())
    
    # Transposing data frame.
    text = text.transpose()
    
    # Sorting data frame according to tf–idf value in descending order.
    text = text.sort_values(by = 0, ascending = False)
    
    # Subsetting top n terms.
    text = text[:n]
    
    # Extracting indices of top n terms.
    idx = text.index.tolist()
    
    # Returning list with n terms with the highest tf–idf value.
    return [vectorizer.get_feature_names()[i] for i in idx]

Your implementation should conform to the following specification:

<strong>keywords</strong> (<em>text</em>, <em>n</em> = 10)

> Returns a list with the $n$ (default value: 10) most salient keywords from the specified text, as measured by their tf–idf value relative to the collection of app descriptions.

Test your implementation by running the following cell:

In [16]:
print(keywords(df["description"][1428]))

# The reason why our result differs slightly from the given result is that there are not exactly 
# 10 terms with the highest tf-idf value. Instead, after sorting the terms descendingly according to their tf-idf-values,
# we could see that the 11th and 12th rank for example showed the same tf-idf-value as the 9th or 10th ranked term.
# Thus, if we would print not the top 10 but for example the top 15, we would show all listed terms below.

['train', 'railway', 'railroad', 'rail', 'chaos', 'crash', 'overcast', 'timetable', 'railyard', 'locomotive']


This should give the following output:

<div class="alert alert-info">
    Please read the General information section on the lab web page before submitting this notebook!
</div>